In [ ]:
# Criação dos Arquivos CSV
!pip install Faker
!pip install requests

# Criação dos Arquivos CSV

from faker import Faker
import json
import numpy
import pandas as pd
import random
import requests

fake = Faker(locale='pt_PT')


def create_csv(fileName: str, values: list):
    df = pd.DataFrame(values)
    df.to_csv(fileName + ".csv", index=False)


# Criação dos Arquivo meals.csv e meals_stock.csv


# Gerando fake data de meals
def get_json_data(url: str):
    response = requests.get(url)
    raw_data = response.content.decode("utf-8")
    json_data = json.loads(raw_data)
    return json_data


def get_random_meal():
    data = get_json_data(
        'http://www.themealdb.com/api/json/v1/1/random.php')
    meal = pd.DataFrame(data['meals']).head(1)
    seller_price = float(fake.pydecimal(
        left_digits=2, right_digits=2, positive=True))
    return {
        'area': meal['strArea'].values[0],
        'category': meal['strCategory'].values[0],
        'imageUrl': meal['strMealThumb'].values[0],
        'instructions': meal['strInstructions'].values[0],
        'mealId': meal['idMeal'].values[0],
        'name': meal['strMeal'].values[0],
        'sellerPrice': seller_price
    }


def mock_meals(meal_qnt: int = 10):
    meals = []
    for _ in range(meal_qnt):
        random_meal = get_random_meal()
        meals.append(random_meal)
    return meals


# Criando arquivo de meals
meals = mock_meals()
create_csv("meals", meals)


# Gerando fake data de meals_stock
def map_meals_stock(meals: list):
    meals_stock_mapped = []
    for meal in meals:
        ideal_quantity = random.randint(10, 100)
        current_quantity = random.randint(ideal_quantity, 200)
        each_meal_profit = 0.7
        cost_price = round(meal["sellerPrice"]*(1 - each_meal_profit), 2)
        meals_stock_mapped.append({"mealId": meal["mealId"], "currentQuantity": current_quantity,
                                   "idealQuantity": ideal_quantity, "costPrice": cost_price})
    return meals_stock_mapped


# Criando arquivo de meals_stock
meals_stock = map_meals_stock(meals)
create_csv("meals_stock", meals_stock)

# Criação dos Arquivos clients.csv e addresses.csv


# Gerando fake dada de address
def mock_address(address_qnt: int):
    addresses = []
    for _ in range(address_qnt):
        address = fake.street_address()
        addresses.append({
            "address": address,
            "alias": fake.text(max_nb_chars=6)
        })
    return addresses


genders = {
    "F": "Feminino",
    "M": "Masculino",
    "NB": "Não Binário",
    "O": "Outro"
}


# Gerando fake dada de clients
def mock_clients(clients_qnt: int):
    clients = []
    for _ in range(clients_qnt):
        gender = numpy.random.choice(
            [genders["M"], genders["F"], genders["NB"], genders["O"]], p=[0.4, 0.5, 0.08, 0.02]) if fake.boolean(chance_of_getting_true=60) else None
        name = fake.name_female() if gender == genders["F"] else fake.name_male(
        ) if gender == genders["M"] else fake.name_nonbinary() if gender == genders["NB"] else fake.name()
        address_qnt = random.randint(1, 3)
        cpf = random.randint(10000000000, 99999999999)
        clients.append({
            "cpf": cpf,
            "name": name,
            "email": fake.unique.email(),
            "birthdate": fake.date_of_birth(maximum_age=100).strftime("%d/%m/%Y"),
            "password": fake.password(length=6),
            "gender": gender,
            "address": mock_address(address_qnt)
        })
    return clients


# Remover o campo address do client
def map_clients(clients: list):
    clients_mapped = []
    for client in clients:
        clients_mapped.append({
            "cpf": client["cpf"],
            "name": client["name"],
            "email": client["email"],
            "birthdate": client["birthdate"],
            "password": client["password"],
            "gender": client["gender"],
        })
    return clients_mapped


# Criando o arquivo clients
clients = mock_clients(100)
create_csv("clients", map_clients(clients))


# Colocando todos os endereços de cada client em uma única lista
def map_address(clients: list):
    address_id = 0
    address_mapped = []
    for client in clients:
        if client["address"]:
            for address in client["address"]:
                address_mapped.append(
                    {"addressId": address_id, "clientId": client["cpf"], **address})
                address_id += 1
    return address_mapped


# Criando o arquivo address
addresses = map_address(clients)
create_csv("addresses", addresses)


# Criação dos Arquivos orders.csv e orders_meals.csv


# Gerando fake data de orders
def mock_orders(clients: list, addresses: list, meals: list, orders_by_client_max: int = 5, meals_by_orders_max: int = 5):
    order_id = 0
    orders = []
    for client in clients:
        orders_quantity = random.randint(0, orders_by_client_max)
        if orders_quantity != 0:
            for _ in range(orders_quantity):
                client_addresses = list(
                    filter(lambda address: address['clientId'] == client["cpf"], addresses))
                client_random_address = random.choice(client_addresses)
                random_30_days_past_date = fake.date_time_between(
                    start_date='-30d', end_date='now')
                random_meals_by_orders_max = random.randint(
                    1, meals_by_orders_max)
                random_meals = random.sample(
                    meals, k=random_meals_by_orders_max)
                total_price = sum(
                    map(lambda meal: meal['sellerPrice'], random_meals))
                orders.append({
                    "orderId": order_id,
                    "clientId": client["cpf"],
                    "addressId": client_random_address["addressId"],
                    "date": random_30_days_past_date,
                    "note": fake.text(),
                    "meals": random_meals,
                    "totalPrice": total_price
                })
                order_id += 1
    return orders


# Remover o campo meals do orders
def map_orders(orders: list):
    orders_mapped = []
    for order in orders:
        orders_mapped.append({
            "orderId": order["orderId"],
            "clientId": order["clientId"],
            "addressId": order["addressId"],
            "date": order["date"],
            "note": order["note"],
            "totalPrice": order["totalPrice"]
        })
    return orders_mapped


# Criando o arquivo orders
orders = mock_orders(clients, addresses, meals)
create_csv("orders", map_orders(orders))


# Gerando fake data de orders_meals
def map_orders_meals(orders: list):
    orders_meals_mapped = []
    for order in orders:
        if order["meals"]:
            for meal in order["meals"]:
                orders_meals_mapped.append({
                    "orderId": order["orderId"],
                    "mealId": meal["mealId"],
                    "quantity": 1,
                    "totalPrice": meal["sellerPrice"]
                })
    return orders_meals_mapped


# Criando o arquivo orders_meals
orders_meals = map_orders_meals(orders)
create_csv("orders_meals", orders_meals)


# Criação do Arquivo stocks_orders.csv

# Gerando fake data do meals_stock
def mock_stocks_orders(meals_stock: list):
    stock_order_id = 0
    stocks_oders = []
    for meal_stock in meals_stock:
        has_stock_order = random.choices([True, False], weights=[0.2, 0.8])
        random_30_days_past_date = fake.date_time_between(
            start_date='-30d', end_date='now')
        quantity = random.randint(10, 100)
        total_price = meal_stock["costPrice"] * quantity
        if has_stock_order:
            stocks_oders.append({
                "stockOrderId": stock_order_id,
                "mealId": meal_stock["mealId"],
                "date": random_30_days_past_date,
                "quantity": quantity,
                "totalPrice": total_price
            })
            stock_order_id += 1
    return stocks_oders


# Criando o arquivo stock_orders
stocks_orders = mock_stocks_orders(meals_stock)
create_csv("stocks_orders", stocks_orders)


# Criação do Arquivo transactions.csv


fixed_transactions = {
    "transactions_maintenance": float(fake.pydecimal(right_digits=2, positive=True, min_value=5000, max_value=10000))*-1,
    "transactions_amount": float(fake.pydecimal(right_digits=2, positive=True, min_value=80000, max_value=160000)),
    "transactions_rental": float(fake.pydecimal(right_digits=2, positive=True, min_value=3000, max_value=6000))*-1,
    "transactions_salaries": float(fake.pydecimal(right_digits=2, positive=True, min_value=10000, max_value=20000))*-1
}


# Gerando fake data de transactions
def mock_transactions(orders, stocks_orders):
    transaction_id = 0
    transactions = []
    for name, value in fixed_transactions.items():
        random_30_days_past_date = fake.date_time_between(
            start_date='-30d', end_date='now')
        transactions.append({
            "transactionId": transaction_id,
            "value": value,
            "date": random_30_days_past_date,
            "name": name,
        })
        transaction_id += 1
    for order in orders:
        random_30_days_past_date = fake.date_time_between(
            start_date='-30d', end_date='now')
        transactions.append({
            "transactionId": transaction_id,
            "value": order["totalPrice"],
            "date": random_30_days_past_date,
            "name": "transactions_order",
            "orderId": order["orderId"]
        })
        transaction_id += 1
    for stock_order in stocks_orders:
        random_30_days_past_date = fake.date_time_between(
            start_date='-30d', end_date='now')
        transactions.append({
            "transactionId": transaction_id,
            "value": stock_order["totalPrice"]*-1,
            "date": random_30_days_past_date,
            "name": "transactions_stock_order",
            "stockOrderId": stock_order["stockOrderId"]
        })
        transaction_id += 1
    return transactions


# Criando o arquivo transactions
transactions = mock_transactions(orders, stocks_orders)
create_csv("transactions", transactions)



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd

########## REQUISITOS FUNCIONAIS ######################
#01 - Verificar disponibilidade de refeições no estoque.
# TRAZENDO A CONSULTA SQL PARA O PYTHON#
#SELECT mealId,
#       CASE WHEN currentQuantity >= 1 THEN TRUE ELSE FALSE END AS quantidade_disponivel
#FROM meal_stock;

# Importar a planilha meal_stock para o DataFrame
df_meal_stock = pd.read_csv('meals_stock.csv')

# Realizar a consulta
df_resultado = df_meal_stock[['mealId', 'currentQuantity']].copy()
df_resultado['quantidade_disponivel'] = df_resultado['currentQuantity'] >= 1

# Exibir o DataFrame resultante
print(df_resultado)


   mealId  currentQuantity  quantidade_disponivel
0   53015               89                   True
1   52848               63                   True
2   52862               99                   True
3   52835              154                   True
4   53048              192                   True
5   52853              127                   True
6   52847              195                   True
7   52917              119                   True
8   53053               90                   True
9   52848              104                   True


In [ ]:
#TRAZENDO A CONSULTA SQL PARA O AMBIENTE PYTHON
#02 - Visualizar a soma de todas as receitas geradas por pedidos no período
#SELECT * FROM transaction
#WHERE value > 0
#AND date BETWEEN '2023-05-01 00:00:00' AND '2023-05-31 00:00:00'
#AND name LIKE '%Order%';"""

from datetime import datetime, timedelta

today = datetime.now()
days_ago = today - timedelta(days=30)

# Importar a planilha transaction para o DataFrame
df_transaction = pd.read_csv('transactions.csv')

# Aplicar as condições de filtro
df_filtered = df_transaction[(df_transaction['value'] > 0) &
                             (df_transaction['date'] >= str(days_ago)) &
                             (df_transaction['date'] <= str(today)) &
                             (df_transaction['name'].str.contains('transactions_order'))]

# Calcular a soma das receitas geradas por pedidos
soma_receitas = df_filtered['value'].sum()

# Exibir a soma das receitas
print('A soma das receitas geradas por pedidos no período é:', soma_receitas)


A soma das receitas geradas por pedidos no período é: 45125.08


In [ ]:
#TRAZENDO A CONSULTA SQL PARA O AMBIENTE PYTHON
#03 Visualizar a quantidade de pedidos feitos no período.
#SELECT COUNT(*) FROM `order` WHERE date BETWEEN '2023-05-01 00:00:00' AND '2023-05-31 00:00:00';

from datetime import datetime, timedelta

today = datetime.now()
days_ago = today - timedelta(days=30)

# Importar a planilha order para o DataFrame
df_order = pd.read_csv('orders.csv')

# Aplicar a condição de filtro
count_pedidos = df_order[(df_order['date'] >= str(days_ago)) & (df_order['date'] <= str(today))].shape[0]

# Exibir a quantidade de pedidos
print('A quantidade de pedidos feitos no período é:', count_pedidos)


A quantidade de pedidos feitos no período é: 242


In [ ]:
#PASSANDO A CONSULTA SQL PARA O PYTHON
#04 Visualizar a quantidade total de clientes.
#SELECT COUNT(*) FROM client;

# Importar a tabela client para o DataFrame
df_client = pd.read_csv('clients.csv')
# Obter a quantidade total de clientes
quantidade_clientes = df_client.shape[0]

# Exibir a quantidade total de clientes
print('A quantidade total de clientes é:', quantidade_clientes)


A quantidade total de clientes é: 100


In [ ]:
# PASSANDO A CONSULTA SQL PARA O PYTHON
#CONSULTA :
# 05 Visualizar a soma dos gastos por categoria: gastos com estoque, gastos gerais.
#SELECT
#    SUM(CASE WHEN value < 0 AND name LIKE '%Salaries%' THEN value ELSE 0 END) as salary_expenses,
#   SUM(CASE WHEN value < 0 AND name LIKE '%Maintanance%' THEN value ELSE 0 END) as maintenance_expenses,
#   SUM(CASE WHEN value < 0 AND name LIKE '%Stock%' THEN value ELSE 0 END) as stock_expenses,
#   SUM(CASE WHEN value < 0 AND name LIKE '%Rental%' THEN value ELSE 0 END) as rental_expenses
#FROM transaction;

# Importar a tabela transaction para o DataFrame
df_transaction = pd.read_csv('transactions.csv')

# Realizar a soma dos gastos por categoria
salary_expenses = df_transaction.loc[(df_transaction['value'] < 0) & (df_transaction['name'].str.contains('transactions_salaries')), 'value'].sum()
maintenance_expenses = df_transaction.loc[(df_transaction['value'] < 0) & (df_transaction['name'].str.contains('transactions_maintenance')), 'value'].sum()
stock_expenses = df_transaction.loc[(df_transaction['value'] < 0) & (df_transaction['name'].str.contains('transactions_stock_order')), 'value'].sum()
rental_expenses = df_transaction.loc[(df_transaction['value'] < 0) & (df_transaction['name'].str.contains('transactions_rental')), 'value'].sum()

# Exibir a soma dos gastos por categoria
print('Gastos com Salários:', salary_expenses)
print('Gastos com Manutenção:', maintenance_expenses)
print('Gastos com Estoque:', stock_expenses)
print('Gastos com Aluguel:', rental_expenses)


Gastos com Salários: -12314.25
Gastos com Manutenção: -9451.13
Gastos com Estoque: -10817.189999999999
Gastos com Aluguel: -5255.58


In [ ]:
# PASSANDO A CONSULTA SQL PARA O PYTHON
#CONSULTA :
#06 Leitura de todos os clientes cadastrados na base de dados.
#SELECT * FROM client;

# Leitura do arquivo CSV com o delimitador '\t'
df_client = pd.read_csv('clients.csv')

# Exibição de todos os clientes cadastrados
print(df_client)


            cpf               name                       email   birthdate  \
0   29401975682     Alexandra Leal      madalena36@example.com  15/11/1996   
1   97709487656     Eduardo Barros   kevinassuncao@example.net  15/01/1972   
2   96548362767        Nair Mendes       anjosjose@example.com  19/05/1950   
3   80376340679      Kelly Correia      leitediego@example.net  23/02/1936   
4   80656461593  Madalena da Pires      marianavaz@example.net  09/02/1940   
..          ...                ...                         ...         ...   
95  44302323111        Inês Borges  jessicaribeiro@example.org  20/06/1969   
96  46501150552   Joaquim Loureiro  carvalhorenato@example.net  02/04/2010   
97  80053286667      Teresa Macedo        tantunes@example.com  20/06/2021   
98  26721825068      António Alves        jmoreira@example.org  14/02/1930   
99  46344473861       Pedro Morais          mmatos@example.com  31/07/1931   

   password     gender  
0    F&8Dw7        NaN  
1    !0S!ej  

In [ ]:
# PASSANDO A CONSULTA SQL PARA O PYTHON
#CONSULTA :
#07 Visualizar refeições mais e menos pedidas por período.
#SELECT om.mealId, m.name, SUM(om.quantity) as total_quantity
#FROM `order_meal` as om
#INNER JOIN `meal` as m
#	ON om.mealId = m.mealId
#INNER JOIN `order` as o
#	ON o.orderId = om.orderId
#WHERE o.date BETWEEN '2023-05-01 00:00:00' AND '2023-05-31 00:00:00'
#GROUP BY om.mealId, m.name
#ORDER BY total_quantity DESC;

from datetime import datetime, timedelta

today = datetime.now()
days_ago = today - timedelta(days=30)

# Importar as tabelas order_meal, meal e order como DataFrames
df_order_meal = pd.read_csv('orders_meals.csv')
df_meal = pd.read_csv('meals.csv')
df_order = pd.read_csv('orders.csv')

# Executar a consulta e obter o resultado
df_result = pd.merge(df_order_meal, df_meal, on='mealId')
df_result = pd.merge(df_result, df_order, on='orderId')
df_result = df_result[(df_result['date'] >= str(days_ago)) & (df_result['date'] <= str(today))]
df_result = df_result.groupby(['mealId', 'name']).agg({'quantity': 'sum'}).reset_index()
df_result = df_result.sort_values(by='quantity', ascending=False)

# Exibir o resultado
print(df_result)


   mealId                          name  quantity
2   52848         Bean & Sausage Hotpot       290
0   52835             Fettucine alfredo        93
1   52847                Pork Cassoulet        77
4   52862       Peach & Blueberry Grunt        76
7   53048              Mee goreng mamak        75
8   53053                  Beef Rendang        72
6   53015            Krispy Kreme Donut        68
3   52853      Chocolate Avocado Mousse        65
5   52917  White chocolate creme brulee        63


In [ ]:
# PASSANDO A CONSULTA SQL PARA O PYTHON
#CONSULTA :
#08 Visualizar os 10 clientes que mais compraram, em valor total por período.
#SELECT c.name, c.CPF, SUM(o.totalPrice) as value_purchases
#FROM `client` as c
#INNER JOIN `order` as o
#	ON c.CPF = o.clientId
#WHERE o.date BETWEEN '2023-05-01 00:00:00' AND '2023-05-31 00:00:00'
#GROUP BY c.name, c.CPF
#ORDER BY value_purchases DESC
#LIMIT 10;

from datetime import datetime, timedelta

today = datetime.now()
days_ago = today - timedelta(days=30)

# Carregar os arquivos CSV
df_client = pd.read_csv('clients.csv')
df_order = pd.read_csv('orders.csv')

# Verificar se a coluna 'CPF' está presente no DataFrame 'df_client'
if 'cpf' not in df_client.columns:
    raise KeyError("A coluna 'CPF' não está presente no DataFrame 'df_client'.")

# Filtrar os dados no período desejado
df_order = df_order[(df_order['date'] >= str(days_ago)) & (df_order['date'] <= str(today))]

# Juntar os DataFrames utilizando a coluna 'CPF' e 'clientId'
df_result = pd.merge(df_client, df_order, left_on='cpf', right_on='clientId', how='inner')

# Agrupar por nome e CPF, somando os valores das compras
df_result = df_result.groupby(['name', 'cpf']).agg({'totalPrice': 'sum'}).reset_index()

# Ordenar pelo valor das compras em ordem decrescente e limitar para os 10 primeiros registros
df_result = df_result.sort_values(by='totalPrice', ascending=False).head(10)

# Exibir o resultado
print(df_result)


                    name          cpf  totalPrice
43         Kevin Azevedo  98782005150     1500.49
48    Lisandro Gonçalves  26515959315     1287.72
60     Mélanie Gonçalves  68883382251     1204.57
74        Soraia Machado  79672230014     1200.73
14  Carminho-Clara Paiva  24472102200     1193.46
21           Erica Lopes  28706668251     1172.50
32            Hugo Rocha  21256900085     1147.47
49            Luna Pires  83761289669     1136.77
18      César Figueiredo  22866363340     1055.38
82       Violeta Pacheco  23268769645     1002.43
